In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'

# 보험 상품 유형, 가입자 성향 다시 분배
##### 가입 시기 나이 80세 이상인 고객을 현실성을 반영해 없애고 다시 반영을 한 것

In [2]:
data = pd.read_csv('C:/baf/data_with_new_products1022.csv')
df = data.copy() 

df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,individual_id,curr_ann_amt,days_tenure,cust_orig_date,date_of_birth,income,has_children,...,city,county,cust_orig_year,cust_orig_month,churn,tenure_years,age_at_orig,premium_to_income,policy_proxy_v2,strategy_type_final
0,0,0,0,2.213000e+11,818.877997,1454.0,2018-12-09,1978-06-23,22500.0,1,...,Kaufman,Kaufman,2018,2018-12,0,3.980835,40.0,0.036395,Variable Whole Life,혼합형
1,1,1,1,2.213001e+11,974.199182,1795.0,2018-01-02,1950-05-30,27500.0,0,...,Grand Prairie,Dallas,2018,2018-01,0,4.914442,67.0,0.035425,Variable Annuity,안정형
2,2,2,2,2.213007e+11,967.375112,4818.0,2009-09-23,1967-07-07,42500.0,0,...,Dallas,Dallas,2009,2009-09,0,13.190965,42.0,0.022762,Variable Universal Life,안정형
3,3,3,3,2.213016e+11,992.409561,130.0,2022-07-25,1969-05-25,125000.0,1,...,Arlington,Tarrant,2022,2022-07,1,0.355921,53.0,0.007939,Variable Universal Life,혼합형
4,4,4,4,2.213016e+11,784.633494,5896.0,2006-10-11,1972-09-25,87500.0,1,...,Fort Worth,Tarrant,2006,2006-10,0,16.142368,34.0,0.008967,Variable Whole Life,혼합형


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1641992 entries, 0 to 1641991
Data columns (total 26 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0.2         1641992 non-null  int64  
 1   Unnamed: 0.1         1641992 non-null  int64  
 2   Unnamed: 0           1641992 non-null  int64  
 3   individual_id        1641992 non-null  float64
 4   curr_ann_amt         1641992 non-null  float64
 5   days_tenure          1641992 non-null  float64
 6   cust_orig_date       1641992 non-null  object 
 7   date_of_birth        1641992 non-null  object 
 8   income               1641992 non-null  float64
 9   has_children         1641992 non-null  int64  
 10  length_of_residence  1641992 non-null  float64
 11  home_market_value    1641992 non-null  object 
 12  age_in_years         1641992 non-null  int64  
 13  home_owner           1641992 non-null  float64
 14  college_degree       1641992 non-null  float64
 15

### 좀 더 구체적이고 현실성있는 로직 기반으로 보험상품, 고객 유형을 부여하기 위해 데이터 분석 및 분포를 다시 진행해 파악

In [4]:
print("="*80)
print("【1】 기본 통계")
print("="*80)
print(f"총 고객 수: {len(df):,}명")
print(f"\n연령: 평균 {df['age_at_orig'].mean():.1f}세, "
      f"중위값 {df['age_at_orig'].median():.1f}세, "
      f"표준편차 {df['age_at_orig'].std():.1f}")
print(f"소득: 평균 ${df['income'].mean():,.0f}, "
      f"중위값 ${df['income'].median():,.0f}")
print(f"보험료: 평균 ${df['curr_ann_amt'].mean():,.0f}, "
      f"중위값 ${df['curr_ann_amt'].median():.0f}")
print(f"보험료/소득 비율: 평균 {df['premium_to_income'].mean():.3f} "
      f"(중위값 {df['premium_to_income'].median():.3f})")

print("\n" + "="*80)
print("【2】 연령대별 분포")
print("="*80)
age_dist = pd.cut(df['age_at_orig'], 
                  bins=[0, 30, 40, 50, 60, 80],
                  labels=['~29세', '30-39세', '40-49세', '50-59세', '60세+'])
print(age_dist.value_counts(normalize=True).sort_index().apply(lambda x: f"{x*100:5.1f}%"))

print("\n" + "="*80)
print("【3】 소득 5분위별 분포")
print("="*80)
income_q = pd.qcut(df['income'], q=5, 
                   labels=['Q1(하위20%)', 'Q2', 'Q3', 'Q4', 'Q5(상위20%)'])
print(income_q.value_counts(normalize=True).sort_index().apply(lambda x: f"{x*100:5.1f}%"))

print("\n" + "="*80)
print("【4】 가입 연도별 분포")
print("="*80)
year_dist = df['cust_orig_year'].value_counts().sort_index()
print(year_dist)
print(f"\n2005-2010: {df[df['cust_orig_year'].between(2005, 2010)].shape[0]:,}명 "
      f"({df[df['cust_orig_year'].between(2005, 2010)].shape[0]/len(df)*100:.1f}%)")
print(f"2011-2015: {df[df['cust_orig_year'].between(2011, 2015)].shape[0]:,}명 "
      f"({df[df['cust_orig_year'].between(2011, 2015)].shape[0]/len(df)*100:.1f}%)")
print(f"2016-2022: {df[df['cust_orig_year'].between(2016, 2022)].shape[0]:,}명 "
      f"({df[df['cust_orig_year'].between(2016, 2022)].shape[0]/len(df)*100:.1f}%)")

print("\n" + "="*80)
print("【5】 자녀 유무 × 연령대 교차표")
print("="*80)
cross_children = pd.crosstab(age_dist, df['has_children'], 
                             normalize='index') * 100
print(cross_children.round(1))

print("\n" + "="*80)
print("【6】 신용 상태 분포")
print("="*80)
print(f"신용 양호(1): {(df['good_credit']==1).sum():,}명 "
      f"({(df['good_credit']==1).sum()/len(df)*100:.1f}%)")
print(f"신용 불량(0): {(df['good_credit']==0).sum():,}명 "
      f"({(df['good_credit']==0).sum()/len(df)*100:.1f}%)")

print("\n" + "="*80)
print("【7】 보험료 부담 수준")
print("="*80)
print(df['premium_to_income'].describe())
print(f"고부담(>5%): {(df['premium_to_income']>0.05).sum()/len(df)*100:.1f}%")
print(f"중부담(2-5%): {df['premium_to_income'].between(0.02,0.05).sum()/len(df)*100:.1f}%")
print(f"저부담(<2%): {(df['premium_to_income']<0.02).sum()/len(df)*100:.1f}%")

print("\n" + "="*80)
print("【8】 해지율")
print("="*80)
print(f"전체 해지율: {df['churn'].mean()*100:.2f}%")
print(f"\n연령대별 해지율:")
print(pd.crosstab(age_dist, df['churn'], normalize='index')[1] * 100)

print("\n" + "="*80)
print("【9】 주요 변수 상관관계 (해지와의 관계)") 
print("="*80)
corr_vars = ['age_at_orig', 'income', 'curr_ann_amt', 'premium_to_income',
             'has_children', 'good_credit', 'length_of_residence','tenure_years', 'churn']
print(df[corr_vars].corr()['churn'].sort_values(ascending=False).round(3))

print("\n" + "="*80)
print("【10】 연령 × 소득 교차 분석")
print("="*80)
income_group = pd.cut(df['income'], 
                      bins=[0, 30000, 50000, 75000, 100000, 200000],
                      labels=['~3만', '3-5만', '5-7.5만', '7.5-10만', '10만+'])
print(pd.crosstab(age_dist, income_group, normalize='index') * 100)

【1】 기본 통계
총 고객 수: 1,641,992명

연령: 평균 44.9세, 중위값 44.0세, 표준편차 13.2
소득: 평균 $82,138, 중위값 $70,000
보험료: 평균 $940, 중위값 $935
보험료/소득 비율: 평균 0.019 (중위값 0.013)

【2】 연령대별 분포
age_at_orig
~29세       14.7%
30-39세     25.5%
40-49세     27.0%
50-59세     19.7%
60세+       13.1%
Name: proportion, dtype: object

【3】 소득 5분위별 분포
income
Q1(하위20%)     22.9%
Q2            19.8%
Q3            34.4%
Q4            14.9%
Q5(상위20%)      8.0%
Name: proportion, dtype: object

【4】 가입 연도별 분포
cust_orig_year
2005    498016
2006     42663
2007     45503
2008     47799
2009     48661
2010     49566
2011     50945
2012     48342
2013     56646
2014     59348
2015     61609
2016     67330
2017     75866
2018     80468
2019     85285
2020     66893
2021     37983
2022    219069
Name: count, dtype: int64

2005-2010: 732,208명 (44.6%)
2011-2015: 276,890명 (16.9%)
2016-2022: 632,894명 (38.5%)

【5】 자녀 유무 × 연령대 교차표
has_children     0     1
age_at_orig             
~29세          39.8  60.2
30-39세        38.0  62.0
40-49세        45.8  54.

# 위 분석 기반 로직 생성
##### 2005년 가입자는 498016명으로 비이상적으로 많은데, 2005년부터 기록한 보험으로 실제로는 누적 데이터일 가능성-> 2005년 이전은 '초기가입자'로 별도그룹화

## 로직 핵심 특징 

#### - 한국 시장 트렌드 반영
    # 2005년 = 초기 누적 데이터 (보수적)
    # 2006-2010 = 연금 중심기
    # 2011-2015 = 종신 확대기  
    # 2016-2022 = 유니버셜 급증기

In [5]:
def assign_insurance_products(df):
    # ========================================
    # STEP 1: 시기별 기본 확률 설정
    # ========================================
    
    # 한국 시장 트렌드 반영
    # 2005년 = 초기 누적 데이터 (보수적)
    # 2006-2010 = 연금 중심기
    # 2011-2015 = 종신 확대기  
    # 2016-2022 = 유니버셜 급증기
    
    period_ratios = {
        2005: {'연금': 0.55, '종신': 0.35, '유니버셜': 0.10},  # 초기 보수적
        2006: {'연금': 0.50, '종신': 0.35, '유니버셜': 0.15},
        2007: {'연금': 0.50, '종신': 0.35, '유니버셜': 0.15},
        2008: {'연금': 0.48, '종신': 0.35, '유니버셜': 0.17},
        2009: {'연금': 0.46, '종신': 0.36, '유니버셜': 0.18},
        2010: {'연금': 0.45, '종신': 0.36, '유니버셜': 0.19},
        2011: {'연금': 0.42, '종신': 0.38, '유니버셜': 0.20},
        2012: {'연금': 0.40, '종신': 0.40, '유니버셜': 0.20},
        2013: {'연금': 0.40, '종신': 0.40, '유니버셜': 0.20},
        2014: {'연금': 0.38, '종신': 0.40, '유니버셜': 0.22},
        2015: {'연금': 0.38, '종신': 0.39, '유니버셜': 0.23},
        2016: {'연금': 0.35, '종신': 0.37, '유니버셜': 0.28},
        2017: {'연금': 0.33, '종신': 0.35, '유니버셜': 0.32},
        2018: {'연금': 0.32, '종신': 0.33, '유니버셜': 0.35},
        2019: {'연금': 0.30, '종신': 0.32, '유니버셜': 0.38},
        2020: {'연금': 0.30, '종신': 0.30, '유니버셜': 0.40},
        2021: {'연금': 0.28, '종신': 0.30, '유니버셜': 0.42},
        2022: {'연금': 0.28, '종신': 0.28, '유니버셜': 0.44}
    }
    
    # ========================================
    # STEP 2: 고객별 상품 부여
    # ========================================
    
    products = []
    
    for idx, row in df.iterrows():
        age = row['age_at_orig']
        income = row['income']
        year = row['cust_orig_year']
        children = row['has_children']
        credit = row['good_credit']
        premium_ratio = row['premium_to_income']
        
        # 해당 연도 기본 확률
        base_prob = period_ratios.get(year, period_ratios[2022])
        
        # 고객 특성 기반 가중치 계산 (합 = 100)
        weight_annuity = base_prob['연금'] * 100
        weight_whole = base_prob['종신'] * 100
        weight_universal = base_prob['유니버셜'] * 100
        
        # 【규칙 1】 연령 조정
        if age >= 50:
            weight_annuity += 15  # 노후 준비
            weight_universal -= 10
            weight_whole -= 5
        elif age < 35:
            weight_universal += 15  # 젊은층 = 유연성
            weight_annuity -= 10
            weight_whole -= 5
        
        # 【규칙 2】 소득 조정 (저소득 유니버셜 제한 강화)
        if income >= 100000:  # 상위 10%
            weight_universal += 10  # 고소득 = 투자성
            weight_annuity -= 5
            weight_whole -= 5
        elif income < 40000:  # 하위 20%
            weight_whole += 18  # 저소득 = 보장 (강화)
            weight_universal -= 16  # 강력 제한 (강화)
            weight_annuity -= 2
        elif income < 50000:  # 하위 30%
            weight_whole += 10
            weight_universal -= 8
            weight_annuity -= 2
        
        # 【규칙 3】 자녀 & 연령 조합
        if children == 1 and 30 <= age <= 55:
            weight_whole += 10  # 사망보장 중요
            weight_annuity -= 5
            weight_universal -= 5
        
        # 【규칙 4】 신용 & 보험료 부담
        if credit == 1 and premium_ratio < 0.02:
            weight_universal += 5  # 여유로운 고객
            weight_annuity -= 3
            weight_whole -= 2
        
        # 음수 방지 (최소값 5)
        weight_annuity = max(5, weight_annuity)
        weight_whole = max(5, weight_whole)
        weight_universal = max(5, weight_universal)
        
        # 확률 정규화
        total = weight_annuity + weight_whole + weight_universal
        prob_annuity = weight_annuity / total
        prob_whole = weight_whole / total
        prob_universal = weight_universal / total
        
        # 확률 기반 샘플링
        product = np.random.choice(
            ['Variable Annuity', 'Variable Whole Life', 'Variable Universal Life'],
            p=[prob_annuity, prob_whole, prob_universal]
        )
        products.append(product)
        
        # 진행률 표시 (10만 건마다)
        if (idx + 1) % 100000 == 0:
            print(f"진행중... {idx+1:,} / {len(df):,} ({(idx+1)/len(df)*100:.1f}%)")
    
    return products

In [6]:
# 1. 상품 유형 부여
products = assign_insurance_products(df)

진행중... 100,000 / 1,641,992 (6.1%)
진행중... 200,000 / 1,641,992 (12.2%)
진행중... 300,000 / 1,641,992 (18.3%)
진행중... 400,000 / 1,641,992 (24.4%)
진행중... 500,000 / 1,641,992 (30.5%)
진행중... 600,000 / 1,641,992 (36.5%)
진행중... 700,000 / 1,641,992 (42.6%)
진행중... 800,000 / 1,641,992 (48.7%)
진행중... 900,000 / 1,641,992 (54.8%)
진행중... 1,000,000 / 1,641,992 (60.9%)
진행중... 1,100,000 / 1,641,992 (67.0%)
진행중... 1,200,000 / 1,641,992 (73.1%)
진행중... 1,300,000 / 1,641,992 (79.2%)
진행중... 1,400,000 / 1,641,992 (85.3%)
진행중... 1,500,000 / 1,641,992 (91.4%)
진행중... 1,600,000 / 1,641,992 (97.4%)


In [7]:
def assign_investment_strategy(df, products):

    strategies = []
    
    for idx, (row, product) in enumerate(zip(df.itertuples(), products)):
        age = row.age_at_orig
        income = row.income
        credit = row.good_credit
        premium_ratio = row.premium_to_income
        
        # 리스크 스코어 계산 (0~100)
        risk_score = 0
        
        # 【요소 1】 연령 (35점 배점) - 조정
        if age < 30:
            risk_score += 30
        elif age < 40:
            risk_score += 24
        elif age < 50:
            risk_score += 18
        elif age < 60:
            risk_score += 12
        else:
            risk_score += 6
        
        # 【요소 2】 소득 (25점 배점) - 조정
        if income >= 100000:
            risk_score += 25
        elif income >= 75000:
            risk_score += 20
        elif income >= 50000:
            risk_score += 15
        else:
            risk_score += 8
        
        # 【요소 3】 상품 유형 (15점 배점) - 약화
        if product == 'Variable Universal Life':
            risk_score += 12  # 투자 목적
        elif product == 'Variable Whole Life':
            risk_score += 8   # 중간
        else:  # Variable Annuity
            risk_score += 5   # 보수적
        
        # 【요소 4】 신용 & 부담률 (10점 배점)
        if credit == 1 and premium_ratio < 0.015:
            risk_score += 10  # 여유로움
        elif credit == 1:
            risk_score += 5
        
        # 【요소 5】 무작위 변동 추가 (±18점으로 확대)
        # 현실에서는 같은 조건이어도 성향이 다름
        risk_score += np.random.randint(-18, 19)
        risk_score = max(0, min(100, risk_score))  # 0-100 범위
        
        # 성향 분류 (목표: 공격 17%, 혼합 60%, 안정 23%)
        if risk_score >= 66:
            strategy = '공격형'
        elif risk_score >= 33:
            strategy = '혼합형'
        else:
            strategy = '안정형'
        
        strategies.append(strategy)
        
        # 진행률 표시
        if (idx + 1) % 100000 == 0:
            print(f"진행중... {idx+1:,} / {len(df):,} ({(idx+1)/len(df)*100:.1f}%)")
    
    return strategies

In [8]:
# 2. 투자 성향 부여
strategies = assign_investment_strategy(df, products)

진행중... 100,000 / 1,641,992 (6.1%)
진행중... 200,000 / 1,641,992 (12.2%)
진행중... 300,000 / 1,641,992 (18.3%)
진행중... 400,000 / 1,641,992 (24.4%)
진행중... 500,000 / 1,641,992 (30.5%)
진행중... 600,000 / 1,641,992 (36.5%)
진행중... 700,000 / 1,641,992 (42.6%)
진행중... 800,000 / 1,641,992 (48.7%)
진행중... 900,000 / 1,641,992 (54.8%)
진행중... 1,000,000 / 1,641,992 (60.9%)
진행중... 1,100,000 / 1,641,992 (67.0%)
진행중... 1,200,000 / 1,641,992 (73.1%)
진행중... 1,300,000 / 1,641,992 (79.2%)
진행중... 1,400,000 / 1,641,992 (85.3%)
진행중... 1,500,000 / 1,641,992 (91.4%)
진행중... 1,600,000 / 1,641,992 (97.4%)


In [9]:
df['policy_proxy_v2'] = products
df['strategy_type_final'] = strategies

In [10]:
def validate_assignment(df, products, strategies):
    print("\n" + "="*80)
    print("【검증】 상품 부여 결과")
    print("="*80)
    
    # 1. 상품 분포
    product_dist = pd.Series(products).value_counts(normalize=True) * 100
    print("\n1. 전체 상품 비율:")
    print(product_dist.round(1))
    
    # 2. 시기별 상품 분포
    print("\n2. 시기별 상품 분포:")
    period_df = df.copy()
    period_df['product'] = products
    
    period_df['period'] = pd.cut(
        period_df['cust_orig_year'],
        bins=[2004, 2005, 2010, 2015, 2022],
        labels=['2005년(초기)', '2006-2010', '2011-2015', '2016-2022']
    )
    
    period_cross = pd.crosstab(
        period_df['period'],
        period_df['product'],
        normalize='index'
    ) * 100
    print(period_cross.round(1))
    
    # 3. 투자 성향 분포
    print("\n3. 투자 성향 분포:")
    strategy_dist = pd.Series(strategies).value_counts(normalize=True) * 100
    print(strategy_dist.round(1))
    
    # 4. 상품 × 성향 교차표
    print("\n4. 상품별 투자 성향 분포:")
    cross_tab = pd.crosstab(
        products,
        strategies,
        normalize='index'
    ) * 100
    print(cross_tab.round(1))
    
    # 5. 연령대별 상품 분포
    print("\n5. 연령대별 상품 분포:")
    age_group = pd.cut(
        df['age_at_orig'],
        bins=[0, 30, 40, 50, 60, 80],
        labels=['~29세', '30-39세', '40-49세', '50-59세', '60세+']
    )
    age_cross = pd.crosstab(age_group, products, normalize='index') * 100
    print(age_cross.round(1))
    
    print("\n" + "="*80)
    print("검증 완료. 비현실적 조합 확인:")
    print("="*80)
    
    # 6. 이상 조합 체크
    df_check = df.copy()
    df_check['product'] = products
    df_check['strategy'] = strategies
    
    # 예: 60세 이상 + 유니버셜 + 공격형 = 비현실적
    weird1 = df_check[
        (df_check['age_at_orig'] >= 60) &
        (df_check['product'] == 'Variable Universal Life') &
        (df_check['strategy'] == '공격형')
    ]
    print(f"• 60세+ & 유니버셜 & 공격형: {len(weird1):,}명 ({len(weird1)/len(df)*100:.2f}%)")
    
    # 예: 저소득 + 유니버셜
    weird2 = df_check[
        (df_check['income'] < 40000) &
        (df_check['product'] == 'Variable Universal Life')
    ]
    print(f"• 저소득(<4만) & 유니버셜: {len(weird2):,}명 ({len(weird2)/len(df)*100:.2f}%)")
    
    print("\n→ 위 비율이 2% 미만이면 양호합니다.")

In [11]:
# 3. 검증
validate_assignment(df, products, strategies)


【검증】 상품 부여 결과

1. 전체 상품 비율:
Variable Annuity           38.2
Variable Whole Life        36.4
Variable Universal Life    25.3
Name: proportion, dtype: float64

2. 시기별 상품 분포:
product    Variable Annuity  Variable Universal Life  Variable Whole Life
period                                                                   
2005년(초기)              50.9                     13.3                 35.8
2006-2010              40.0                     22.3                 37.7
2011-2015              34.1                     24.4                 41.6
2016-2022              29.5                     36.4                 34.2

3. 투자 성향 분포:
혼합형    66.0
공격형    17.6
안정형    16.4
Name: proportion, dtype: float64

4. 상품별 투자 성향 분포:
col_0                     공격형   안정형   혼합형
row_0                                    
Variable Annuity         10.4  23.0  66.6
Variable Universal Life  32.0   5.1  62.9
Variable Whole Life      15.1  17.3  67.7

5. 연령대별 상품 분포:
col_0        Variable Annuity  Variable Universal Life  

In [12]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,individual_id,curr_ann_amt,days_tenure,cust_orig_date,date_of_birth,income,has_children,...,city,county,cust_orig_year,cust_orig_month,churn,tenure_years,age_at_orig,premium_to_income,policy_proxy_v2,strategy_type_final
0,0,0,0,2.213000e+11,818.877997,1454.0,2018-12-09,1978-06-23,22500.0,1,...,Kaufman,Kaufman,2018,2018-12,0,3.980835,40.0,0.036395,Variable Whole Life,혼합형
1,1,1,1,2.213001e+11,974.199182,1795.0,2018-01-02,1950-05-30,27500.0,0,...,Grand Prairie,Dallas,2018,2018-01,0,4.914442,67.0,0.035425,Variable Annuity,안정형
2,2,2,2,2.213007e+11,967.375112,4818.0,2009-09-23,1967-07-07,42500.0,0,...,Dallas,Dallas,2009,2009-09,0,13.190965,42.0,0.022762,Variable Annuity,혼합형
3,3,3,3,2.213016e+11,992.409561,130.0,2022-07-25,1969-05-25,125000.0,1,...,Arlington,Tarrant,2022,2022-07,1,0.355921,53.0,0.007939,Variable Annuity,혼합형
4,4,4,4,2.213016e+11,784.633494,5896.0,2006-10-11,1972-09-25,87500.0,1,...,Fort Worth,Tarrant,2006,2006-10,0,16.142368,34.0,0.008967,Variable Universal Life,공격형
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641987,1641987,1668789,1668789,2.213007e+11,1259.900413,803.0,2020-09-20,1967-07-07,27500.0,1,...,Dallas,Dallas,2020,2020-09,0,2.198494,53.0,0.045815,Variable Annuity,혼합형
1641988,1641988,1668790,1668790,2.213015e+11,604.096865,3261.0,2013-12-28,1945-07-01,87500.0,0,...,Richardson,Dallas,2013,2013-12,0,8.928131,68.0,0.006904,Variable Annuity,혼합형
1641989,1641989,1668791,1668791,2.213026e+11,1255.570597,3403.0,2013-08-08,1981-09-22,125000.0,0,...,Rowlett,Dallas,2013,2013-08,0,9.316906,31.0,0.010045,Variable Annuity,공격형
1641990,1641990,1668792,1668792,2.213008e+11,1104.105051,1258.0,2019-06-23,1970-07-26,87500.0,1,...,Keller,Tarrant,2019,2019-06,0,3.444216,48.0,0.012618,Variable Universal Life,혼합형
